# Markov transitioner. Attempt 1.

This notebook uses the markov models implemented in the markovmodel.py library to find the most likely path through a transitioning network.

That is, if we have, say, 10 steps, and are using two MMs, MM1 and MM2, then the first step will use a MM that's 90% MM1 and 10% MM2, then 80% MM1 and 20% MM2 and so on.

The aim is to try to find smooth transitions between sequential data sources.

In [1]:
import markovmodels as mm

Now, let's define a random MM which uses the states in {'a', 'b', 'c', 'd', 'e'}:

In [2]:
from random import randint

In [3]:
states_ls=['a', 'b', 'c', 'd', 'e']

test1_mm=mm.MarkovModel([(states_ls[randint(0, len(states_ls)-1)],
                          states_ls[randint(0, len(states_ls)-1)])
                         for i in range(20)],
                        [states_ls[randint(0, len(states_ls)-1)]
                         for i in range(10)]
                        )

And another random MM which uses the states in {'c', 'd', 'e', 'f', 'g'}:

In [4]:
states_ls=['c', 'd', 'e', 'f', 'g']

test2_mm=mm.MarkovModel([(states_ls[randint(0, len(states_ls)-1)],
                          states_ls[randint(0, len(states_ls)-1)])
                         for i in range(20)],
                        [states_ls[randint(0, len(states_ls)-1)]
                         for i in range(10)]
                        )

In [5]:
test2_mm.transitionMatrix_ar

array([[ 0.2       ,  0.        ,  0.2       ,  0.4       ,  0.2       ],
       [ 0.        ,  0.2       ,  0.2       ,  0.        ,  0.6       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.14285714,  0.14285714,  0.        ,  0.57142857,  0.14285714]])

So we can go from, say, state 'a' to state 'd' in MM1:

In [6]:
# Let's try it in 5 steps
test1_mm.apply(['a'], 5).most_likely_path('d')

['a', 'a', 'a', 'a', 'c', 'd']

and state 'd' to state 'g' in MM2:

In [7]:
# Let's try it in 5 steps
test2_mm.apply(['d'], 5).most_likely_path('g')

['d', 'e', 'g', 'f', 'd', 'g']

but 'a' is not in MM2, and 'g' is not in MM1, so we need to combine them. And we can do that with a number of steps. Let's assume we're using 10.

Start with a 100% of MM1:

In [8]:
merge_mm=mm.merge(test1_mm, test2_mm, 1)
s1=merge_mm.apply(['a'])
s1.get_current_state_distribution()

array([ 0.4,  0.2,  0.2,  0. ,  0.2,  0. ,  0. ])

Next do 90% of MM1:

In [9]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.9)
s2=merge_mm.apply(s1)
s2.get_current_state_distribution()

array([ 0.16 ,  0.08 ,  0.08 ,  0.06 ,  0.08 ,  0.008,  0.02 ])

Next do 80% of MM1:

In [10]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.8)
s3=merge_mm.apply(s2)
s3.get_current_state_distribution()

array([ 0.064     ,  0.032     ,  0.032     ,  0.02133333,  0.032     ,
        0.01142857,  0.016     ])

Next do 70% of MM1:

In [11]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.7)
s4=merge_mm.apply(s3)
s4.get_current_state_distribution()

array([ 0.0256    ,  0.0128    ,  0.0128    ,  0.01142857,  0.0128    ,
        0.00914286,  0.0096    ])

and so on...

In [12]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.6)
s5=merge_mm.apply(s4)
s5.get_current_state_distribution()

array([ 0.01024   ,  0.00512   ,  0.00512   ,  0.00914286,  0.00512   ,
        0.00548571,  0.00512   ])

In [13]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.5)
s6=merge_mm.apply(s5)
s6.get_current_state_distribution()

array([ 0.004096  ,  0.002048  ,  0.002048  ,  0.00548571,  0.002048  ,
        0.00292571,  0.00274286])

In [14]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.4)
s7=merge_mm.apply(s6)
s7.get_current_state_distribution()

array([ 0.0016384 ,  0.0008192 ,  0.0008192 ,  0.00292571,  0.001024  ,
        0.00156735,  0.00197486])

In [15]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.3)
s8=merge_mm.apply(s7)
s8.get_current_state_distribution()

array([ 0.00065536,  0.00032768,  0.00032768,  0.00156735,  0.00055589,
        0.00112849,  0.0012288 ])

In [16]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.2)
s9=merge_mm.apply(s8)
s9.get_current_state_distribution()

array([ 0.00026214,  0.00013107,  0.00017554,  0.00112849,  0.00030302,
        0.00070217,  0.00075233])

In [17]:
merge_mm=mm.merge(test1_mm, test2_mm, 0.1)
s10=merge_mm.apply(s9)
s10.get_current_state_distribution()

array([  1.04857600e-04,   5.24288000e-05,   1.07475219e-04,
         7.02171429e-04,   2.21936327e-04,   4.29900875e-04,
         6.09384490e-04])

In [18]:
merge_mm=mm.merge(test1_mm, test2_mm, 0)
s11=merge_mm.apply(s10)
s11.get_current_state_distribution()

array([  0.00000000e+00,   0.00000000e+00,   8.70549271e-05,
         4.29900875e-04,   1.40434286e-04,   3.48219708e-04,
         4.21302857e-04])

In [19]:
s11.get_current_state_distribution()

array([  0.00000000e+00,   0.00000000e+00,   8.70549271e-05,
         4.29900875e-04,   1.40434286e-04,   3.48219708e-04,
         4.21302857e-04])

Now, if we get the path to 'g', then we'd hope to see more of the MM1 states in the first half, and more of the MM2 states in the second half:

In [20]:
s11.most_likely_path('g')

['a', 'a', 'a', 'a', 'e', 'g', 'f', 'd', 'g', 'f', 'd', 'g']

Wow... that might have actually worked.

So... say we want to make the transition in 10 steps. Actually, we want 11, for the cases [0, 0.1, 0.2, ..., 0.9, 1.0]

So for *n* steps, want steps:

In [21]:
numSteps_i=5

[1-(x/numSteps_i) for x in range(numSteps_i+1)]

[1.0, 0.8, 0.6, 0.4, 0.19999999999999996, 0.0]

In [22]:
[i for i in reversed([x/numSteps_i for x in range(numSteps_i)])]
    

[0.8, 0.6, 0.4, 0.2, 0.0]

And to calculate the path, want:

In [23]:
# Starting with a MM in test1_mm
# A second MM in test2_mm
# A number of steps in numSteps_i

numSteps_i=10

# Start with an initial state, from 100% MM1. Here, use 'a':

merged_mm=mm.merge(test1_mm, test2_mm, 1)
state_ms=merged_mm.apply(['a'])

# Now do the rest of the cases:

for weighting in reversed([x/numSteps_i for x in range(numSteps_i)]):
    merged_mm=mm.merge(test1_mm, test2_mm, weighting)
    state_ms=merged_mm.apply(state_ms)

# And find most likely path to 'g':
state_ms.most_likely_path('g')

['a', 'a', 'a', 'a', 'e', 'g', 'f', 'd', 'g', 'f', 'd', 'g']

In [24]:
[i for i in reversed([x/numSteps_i for x in range(numSteps_i)])]

[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0]

In [25]:
test1_mm.transitionMatrix_ar

array([[ 0.4       ,  0.2       ,  0.2       ,  0.        ,  0.2       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.16666667,  0.16666667,  0.33333333,  0.33333333],
       [ 0.33333333,  0.16666667,  0.33333333,  0.        ,  0.16666667],
       [ 0.66666667,  0.        ,  0.33333333,  0.        ,  0.        ]])

OK, that seems to be working...

Now to try it on some real data.